# Use Requests to Get Article URLs
This notebook queries the New York Time's [Article Search API](http://developer.nytimes.com/article_search_v2.json) for articles containing the word "terror" and gathers the article URLs.

In [ ]:
import json
import requests
import time
import pickle

In [ ]:
def get_request(query):
    r = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?{0}&api-key={1}'.format(query,API_key))
    time.sleep(1)
    
    return json.loads(r.text)


def get_paginated_request(query,hits):
    results = {}
    for page in range(1, int(np.ceil(hits/10)) + 1):
        try:
            query += '&page={}'.format(page)
            r = requests.get('https://api.nytimes.com/svc/search/v2/articlesearch.json?{0}&api-key={1}'.format(query,API_key))
            time.sleep(5)

            results[page] = json.loads(r.text)
        except:
            pass
        
    return results


def collect_urls(results):
    urls = []

    for page in results.keys():
        try:
            for doc in results[page]['response']['docs']:
                if doc['source'] == 'The New York Times':
                    urls.append(doc['web_url']) 
        except:
            pass
        
    return urls


def query_year(year,term):
    query = 'q={0}&begin_date={1}0101&end_date={1}1231'.format(term,year)
    hits = get_request(query)['response']['meta']['hits']
    page_results = get_paginated_request(query,hits)
    urls = collect_urls(page_results)
    pickle.dump(urls, open('../data/pickle_jar/{0}_{1}_urls.p'.format(year,term),'wb'))
    
    return None

### Query API for articles and collect URLs

In [ ]:
for year in range(1993,2018):
    query_year(year,'terror')